<a href="https://colab.research.google.com/github/rklpsia/python/blob/main/SQL_YumaevaRegina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

# Создание или подключение к базе данных
conn = sqlite3.connect("library.db")
# Создание курсора, который позволяет выполнять SQL-запросы к базе данных.
cursor = conn.cursor()
# Выполняет SQL-запрос, переданный в виде строки.
#PRIMARY KEY — указывает, что это основной ключ таблицы.
#AUTOINCREMENT — значение автоматически увеличивается при добавлении новой строки.
#NOT NULL — столбец не может быть пустым,обязательное поле.
cursor.execute("""
               CREATE TABLE IF NOT EXISTS books (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               title TEXT NOT NULL,
               author TEXT NOT NULL,
               year_published INTEGER NOT NULL,
               genre TEXT
)
""")

#Добавление записей в таблицу
book = [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic")
]
cursor.executemany("INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?,?)", book)
# Сохранение изменений
conn.commit()
#Выборка всех записей из таблицы
cursor.execute("SELECT * FROM books")
rows = cursor.fetchall()  # Возвращает список кортежей

for row in rows:
    print(row)

#Фильтрация по году выпуска
cursor.execute("SELECT * FROM books WHERE year_published > 1950")
rows = cursor.fetchall()

for row in rows:
    print(row)

#Выбор с условием
#LIKE — оператор для поиска по шаблону. 'T%' — шаблон, где:T — первая буква названия должна быть T. a % — любое количество любых символов после T.
cursor.execute("SELECT * FROM books WHERE title like 'T%'")
rows = cursor.fetchall()

for row in rows:
    print(row)

#Сортировка по возрастанию года публикации
cursor.execute("SELECT * FROM books ORDER BY year_published ASC")
rows = cursor.fetchall()

for row in rows:
    print(row)

#Первые две книги, в сортировке по названию
cursor.execute("SELECT * FROM books ORDER BY title LIMIT 2")
rows = cursor.fetchall()

for row in rows:
    print(row)

#Общее количество книг в таблице
cursor.execute("SELECT COUNT(*) FROM books")
result = cursor.fetchone()  # fetchone() возвращает кортеж с одним значением

# Вывод количества строк
print(f"Количество книг в таблице: {result[0]}")
#ИЛИ
query = """
SELECT COUNT(*) AS count
FROM books
"""

df = pd.read_sql(query, conn)
df

#посчитать количество книг для каждого жанра
cursor.execute("""
    SELECT genre, COUNT(*) AS count
    FROM books
    GROUP BY genre
""")
results = cursor.fetchall()
for row in results:
    genre, count = row
    print(f"Жанр: {genre}, Количество книг: {count}")

#обновить год публикации книги "1984" на 1948
cursor.execute("""
UPDATE books
SET year_published = 1948
WHERE title = "1984"
""")
conn.commit()

#Удаление книги
cursor.execute("DELETE FROM books WHERE title = ?", ("The Great Gatsby",))
conn.commit()

#Создать индекс для столбца автор
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_books_author
ON books (author);
""")
conn.commit()
#Проверить есть ли index
query = """
SELECT name
FROM sqlite_master
WHERE type='index' AND name='idx_books_author'
"""
results = cursor.fetchall()
for row in results:
    print(row)

#Создание триггера, который записывает изменения в таблицу при добавлении новой книги
cursor.execute('''
CREATE TABLE IF NOT EXISTS books_logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    action TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
);
''')
conn.commit()
cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_books_changes
AFTER INSERT ON books
FOR EACH ROW
BEGIN
    INSERT INTO books_logs (book_id, action)
    VALUES (NEW.id, 'INSERT');
END;
''')
cursor.execute("""
INSERT INTO books (title, author, year_published, genre)
VALUES (?, ?, ?, ?);
""", ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"))

query = """
SELECT *
FROM books_logs
"""

df = pd.read_sql(query, conn)
df


#Создание транзакции, которая выполняет след.действия
#1. Добавляет новую книгу. 2. Обновляет год публикации другой книги 3.Отменяет изменения, если есть ошибка
try:
    cursor.execute('''
INSERT INTO books (title, author, year_published, genre)
VALUES ('Crime and punishment', 'Dostoevsky', 1865, 'Classic');
''')

    cursor.execute('''
UPDATE books
SET year_published = 1966
WHERE title = '19842';
''')
    conn.commit()
except sqlite3.Error as e:
    print(f"\nОшибка транзакции: {e}. Изменения отменены.")
    conn.rollback()

finally:
    cursor.execute("""
        SELECT * FROM books
    """)
    rows = cursor.fetchall()
    for row in rows:
        print(row)


#Пользовательская функция, которая вычисляет возраст книги(текущий год - год публикации)
from datetime import datetime

def calculate_book_age(year_published):
    current_year = datetime.now().year
    return current_year - year_published

conn.create_function('book_age', 1, calculate_book_age)

cursor = conn.cursor()

cursor.execute('''
SELECT title, year_published, book_age(year_published)
FROM books
''')
books_with_age = cursor.fetchall()

for book in books_with_age:
    print(f"{book[0]} ({book[1]}): {book[2]} лет")

#Создать представление, которое содержит книги, опубликованные после 1950
cursor.execute('''
CREATE VIEW IF NOT EXISTS modern_books AS
SELECT * FROM books WHERE year_published > 1950
''')
conn.commit()


cursor.execute('SELECT * FROM modern_books')
modern_books = cursor.fetchall()


for book in modern_books:
    print(book)


#Удалить таблицу полностью
#cursor.execute("""
#    DROP TABLE books
#""")
#conn.commit()


#Композитный индекс. создание для столбцов автор и год публикации
cursor.execute('''
CREATE INDEX IF NOT EXISTS idx_author_year
ON books (author, year_published)
''')
conn.commit()

# Закрытие соединения
conn.close()

#Удаление базы данных
import os
if os.path.exists('library.db'):
    os.remove('library.db')
    print("\nФайл базы данных library.db удален")
else:
    print("\nФайл базы данных library.db не существует")

